This week we'll be building up your understanding of molecular simulation techniques, starting with MD simulations and moving to MC simulations. 

As we do, we'll be *fading in* more complexity. 


At it's core, running a molecular simulation consists of 5 parts:
1. "Turning on" the simulation software
1. Creating an initial configuration 
1. Define the rules of our model
1. Performing a number of "steps" forward in time
1. Analyzing the simulation data

These steps are not always performed sequentially, but they're always there.

In our first simulation example we're going to:
1. Turn on hoomd-blue, a simulation engine that works nicely in python
1. Load in a configuration of randomly-packed spheres that has been created for us
1. Define Lennard-Jones interaction rules and an NVT ensemble
    1. Bonus: We're going to set the initial velocities, too
1. Step forward 10000 steps.
1. Visualize the last snapshot.



In [1]:
import hoomd

In [2]:
# Attach to CPU and create simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_gsd(filename='random.gsd') #N and V are set here

#Potential and integrator setup
integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer = 0.4)
lj_potential = hoomd.md.pair.LJ(nlist=cell)
lj_potential.params[('A','A')] = dict(epsilon=1,sigma=1)
lj_potential.r_cut[('A','A')]=2.5
nvt = hoomd.md.methods.NVT(kT=1.5,filter=hoomd.filter.All(),tau=1.)
integrator.forces.append(lj_potential)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

# Set the simulation state 
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

# Run  the simulation  (a few seconds)
sim.run(1e4)

In [3]:
#And finally let's visualize the output (usually ~4 seconds, but sometime slow on /shared?)
import mbuild
hoomd.write.GSD.write(state=sim.state, filename='final.gsd', mode='wb') 
final = mbuild.load("final.gsd")


In [4]:
#del view
view = final.visualize()
view.setStyle({"sphere": {"scale": 3., "colorscheme": {"A":"red"} }})
view.addBox({"center":{"x":0,"y":0,"z":0},"dimensions": {"w":10*final.box.Lx,"h":10*final.box.Ly,"d":10*final.box.Lz},"color":'gray',"opacity":0.6})

/opt/conda/lib/python3.10/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<A pos=([0.8647 0.5839 4.0791]), 0 bonds, id: 22851194966640>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol